# Example: Routing using different option parameters

This examples gives you an idea how to use the option and extra info parameter within the direction module of Openrouteservice to receive your individual routing experience.

In [1]:
import folium
from folium import Map, Marker, FeatureGroup, LayerControl

from openrouteservice import client

We are working for a transportation company and are receiving a job to carry goods from Munich to Budapest. Because we have to drive the way in the future quite often, it would be great to get some informations about different routing options for HGV (heavy goods vehicle).

Using the parameter 'extra_info', the tollway data comes as waypoints and not as coordinates. The following function 'TollroadSections' will pick out the rollroad sections from the tollfree sections and convert the separated waypoints to usable coordinates.

In [2]:
# GeoJSON style function
def style_function(color):
    return lambda feature: dict(color=color)


# Request tollroad sections
def TollroadSections(route_with_extrainfo, route_normal_request):
    toll_coords = []
    for value in route_with_extrainfo['routes']:
        toll_values = [value['extras']['tollways']['values']]

        # Tollroad parts are just given as waypoints, why they have to be combined with coordinates
        route_kombi_dict = [toll_values, route_normal_request]

        for data in route_kombi_dict:
            for value in route_kombi_dict[0][0]:
                if 1 in value: # 1 = tollroad, 0 = no tollroad
                    vehicle_value = [value[0:2]]

                    for i,j in vehicle_value:
                        coords = [coords['geometry']['coordinates'][i:j] for coords in route_kombi_dict[1]['features']]
                        toll_coords.append(coords)
    return toll_coords

For the beginning we will request the usual route for HGV from Munich to Budapest. Here we will already mark the tollroad sections using the just defined function. To receive the needed extra tollway info, the `{'format_out': 'geojson'}` paramter has to be turned off, because the data does not come in GeoJson format.

In [3]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # Provide your personal API key
clnt = client.Client(key=api_key) # Create client with api key
map1 = folium.Map(tiles='Stamen Toner', location=([48.57479, 15.46875]), zoom_start=7) # Set up folium map

# Request fastest route from Munich to Budapest
params_route = {'coordinates': [[11.544056, 48.151543], [19.092865, 47.514882]],
                'profile': 'driving-hgv',
                'format_out': 'geojson', 
                'geometry': 'true',
                'preference': 'fastest',
                'instructions': 'false',
                'extra_info': {'tollways'}}

# Request normal Route
route_normal = clnt.directions(**params_route)
folium.features.GeoJson(data=route_normal,
                        name='Tollroute',
                        style_function=style_function('#0000ff'),
                        overlay=True
                       ).add_to(map1)


# Request extra tollway info 
tollsection_group = FeatureGroup(name='Toll sections')

del params_route['format_out'] # To receive extra info, {'format_out': 'geojson'} has to be turned off
route_extra = clnt.directions(**params_route)
tollsection_normal = TollroadSections(route_extra, route_normal) # Request tollroad sections with defined function

for part in tollsection_normal:
    folium.PolyLine([(y,x) for x,y in part[0]], color='#cc0000').add_to(tollsection_group) # Tollroad sections
tollsection_group.add_to(map1)

print('Total distance using tollroads: {:.2f} km\nDriven Distance on tollroad sections: {:.2f} km\nTotal duration: {:.2f} hours'.format(route_extra['routes'][0]['summary']['distance']/1000,
                                                                                                                                         route_extra['routes'][0]['extras']['tollways']['summary'][0]['distance']/1000,
                                                                                                                                         route_extra['routes'][0]['summary']['duration']/3600))
#map1

Total distance using tollroads: 684.02 km
Driven Distance on tollroad sections: 472.22 km
Total duration: 9.35 hours


Okay that looks like a lot tollroad to drive! How about the same route, but avoiding the tollroad and just driving tollfree routes?

In [4]:
# Request route avoiding tollroads
params_route['format_out'] = 'geojson' # Add again the 'format_out' parameter
params_route['options'] = {'avoid_features': 'tollways'} # Define the option paramter

route_tollfree = clnt.directions(**params_route)
folium.features.GeoJson(data=route_tollfree,
                        name='Tollfree route',
                        style_function=style_function('#008000'),
                        overlay=True
                       ).add_to(map1)

print('Total distance using tollfree roads: {:.2f} km\nTotal duration: {:.2f} hours'.format(route_tollfree['features'][0]['properties']['summary'][0]['distance']/1000,
                                                                                        route_tollfree['features'][0]['properties']['summary'][0]['duration']/3600))
map1

Total distance using tollfree roads: 704.62 km
Total duration: 11.76 hours


Now let's have a look, maybe the routing looks better, if we avoid the austrian border and driving via Czech Republic.

In [5]:
# Request extra tollway info and avoiding austrian border
params_route['options'] = {'avoid_countries': '11'} # Avoid austrian border
route_avoidaustria = clnt.directions(**params_route)
folium.features.GeoJson(data=route_avoidaustria,
                        name='Tollroute, avoid austrian border',
                        style_function=style_function('#1aa3ff'),
                        overlay=True
                       ).add_to(map1)

# Request extra tollway info 
del params_route['format_out']
route_avoidaustria_extra = clnt.directions(**params_route)
tollsection_avoidaustria = TollroadSections(route_avoidaustria_extra, route_avoidaustria) # Defined function

for part in tollsection_avoidaustria:
    folium.PolyLine([(y,x) for x,y in part[0]], color='#cc0000').add_to(tollsection_group) # Tollroad sections
tollsection_group.add_to(map1)

print('Total distance using tollroads and avoiding the austrian border: {:.2f} km\nDriven distance on tollroad sections: {:.2f} km\nTotal duration: {:.2f} hours'.format(route_avoidaustria_extra['routes'][0]['summary']['distance']/1000,
                                                                                                                                              route_avoidaustria_extra['routes'][0]['extras']['tollways']['summary'][1]['distance']/1000,
                                                                                                                                             route_avoidaustria_extra['routes'][0]['summary']['duration']/3600))

#map1

Total distance using tollroads and avoiding the austrian border: 819.89 km
Driven distance on tollroad sections: 363.38 km
Total duration: 12.08 hours


In [6]:
# Request tollfree route and avoiding austrian border
params_route['format_out'] = 'geojson'
params_route['options'] = {'avoid_features': 'tollways',
                           'avoid_countries': '11'} # Avoid austrian border

route_avoidaustria_tollfree = clnt.directions(**params_route)
folium.features.GeoJson(data=route_avoidaustria_tollfree,
                        name='Tollfree route, avoid austrian border',
                        style_function=style_function('#5cd65c'),
                        overlay=True
                       ).add_to(map1)

print('Total distance using tollfree roads and avoiding the austrian border: {:.2f} km\nTotal duration: {:.2f} hours'.format(route_avoidaustria_tollfree['features'][0]['properties']['summary'][0]['distance']/1000,
                                                                                        route_avoidaustria_tollfree['features'][0]['properties']['summary'][0]['duration']/3600))

map1.add_child(folium.map.LayerControl())
map1

Total distance using tollfree roads and avoiding the austrian border: 767.46 km
Total duration: 13.06 hours


To get a better overview here a summary of all reveived informations:

In [12]:
print('Information for using tollroads and driving via Austria:\nTotal distance using tollroads: {:.2f} km\nDriven Distance on tollroad sections: {:.2f} km\nTotal duration: {:.2f} hours\n'.format(route_extra['routes'][0]['summary']['distance']/1000,
                                                                                                                                         route_extra['routes'][0]['extras']['tollways']['summary'][0]['distance']/1000,
                                                                                                                                         route_extra['routes'][0]['summary']['duration']/3600))

print('Information for only using tollfree roads and driving via Austria:\nTotal distance using tollfree roads: {:.2f} km\nTotal duration: {:.2f} hours\n'.format(route_tollfree['features'][0]['properties']['summary'][0]['distance']/1000,
                                                                                        route_tollfree['features'][0]['properties']['summary'][0]['duration']/3600))

print('Information for using tollroads and avoiding the austrian border:\nTotal distance: {:.2f} km\nDriven distance on tollroad sections: {:.2f} km\nTotal duration: {:.2f} hours\n'.format(route_avoidaustria_extra['routes'][0]['summary']['distance']/1000,
                                                                                                                                              route_avoidaustria_extra['routes'][0]['extras']['tollways']['summary'][1]['distance']/1000,
                                                                                                                                             route_avoidaustria_extra['routes'][0]['summary']['duration']/3600))

print('Information for only using tollfree roads and avoiding the austrian border:\nTotal distance: {:.2f} km\nTotal duration: {:.2f} hours'.format(route_avoidaustria_tollfree['features'][0]['properties']['summary'][0]['distance']/1000,
                                                                                        route_avoidaustria_tollfree['features'][0]['properties']['summary'][0]['duration']/3600))


Information for using tollroads and driving via Austria:
Total distance using tollroads: 684.02 km
Driven Distance on tollroad sections: 472.22 km
Total duration: 9.35 hours

Information for only using tollfree roads and driving via Austria:
Total distance using tollfree roads: 704.62 km
Total duration: 11.76 hours

Information for using tollroads and avoiding the austrian border:
Total distance: 819.89 km
Driven distance on tollroad sections: 363.38 km
Total duration: 12.08 hours

Information for only using tollfree roads and avoiding the austrian border:
Total distance: 767.46 km
Total duration: 13.06 hours


Seems like the second routing option with just using tollfree roads but driving via Austria is a good option to choose. We save some money and it is still an acceptable duration. 